In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import os


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
# Hyperparameters
batch_size = 128
image_size = 64
nz = 100  # Size of latent vector
ngf = 64  # Generator feature maps
ndf = 64  # Discriminator feature maps
epochs = 10
lr = 0.0002
beta1 = 0.5  # Adam optimizer beta1 value

In [ ]:
dataroot = "./Dataset_DCGANS"
dataset = dset.ImageFolder(root=dataroot, transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

generator = Generator().to(device)

In [ ]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

discriminator = Discriminator().to(device)


In [ ]:
# Loss and Optimizer
criterion = nn.BCELoss()
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# Training Loop
for epoch in range(epochs):
    for i, data in enumerate(dataloader):
        real_images = data[0].to(device)
        batch_size = real_images.size(0)
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)

        # Train Discriminator
        discriminator.zero_grad()
        output_real = discriminator(real_images).view(-1, 1)
        loss_real = criterion(output_real, real_labels)

        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake_images = generator(noise)
        output_fake = discriminator(fake_images.detach()).view(-1, 1)
        loss_fake = criterion(output_fake, fake_labels)

        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizerD.step()

        # Train Generator
        generator.zero_grad()
        output_fake = discriminator(fake_images).view(-1, 1)
        loss_G = criterion(output_fake, real_labels)
        loss_G.backward()
        optimizerG.step()

        if i % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {i}/{len(dataloader)} Loss D: {loss_D.item()}, Loss G: {loss_G.item()}")

    # Save sample images after each epoch
    with torch.no_grad():
        fake_images = generator(torch.randn(64, nz, 1, 1, device=device)).cpu()
    vutils.save_image(fake_images, f"./generated_epoch_{epoch}.png", normalize=True)

print("Training Complete!")

Epoch [0/10] Batch 0/1583 Loss D: 1.3705850839614868, Loss G: 2.437617301940918
Epoch [0/10] Batch 100/1583 Loss D: 0.07309041917324066, Loss G: 6.464035987854004
Epoch [0/10] Batch 200/1583 Loss D: 0.9527121186256409, Loss G: 5.781160354614258
Epoch [0/10] Batch 300/1583 Loss D: 1.016509771347046, Loss G: 1.3346107006072998
Epoch [0/10] Batch 400/1583 Loss D: 1.6353894472122192, Loss G: 7.176621437072754
Epoch [0/10] Batch 500/1583 Loss D: 0.5331792831420898, Loss G: 2.8078701496124268
Epoch [0/10] Batch 600/1583 Loss D: 0.3280215859413147, Loss G: 3.406755208969116
Epoch [0/10] Batch 700/1583 Loss D: 0.7408376336097717, Loss G: 4.491232872009277
Epoch [0/10] Batch 800/1583 Loss D: 0.6432062387466431, Loss G: 3.3877859115600586
Epoch [0/10] Batch 900/1583 Loss D: 1.204240322113037, Loss G: 1.796863317489624
Epoch [0/10] Batch 1000/1583 Loss D: 0.6074163913726807, Loss G: 3.6432042121887207
Epoch [0/10] Batch 1100/1583 Loss D: 0.9701594114303589, Loss G: 2.1877872943878174
Epoch [0/10]